# Задание 8
Даны результаты работы трех машинных переводчиков (N,P,Z) на небольших выборках переводов с английского на немецкий, с английского на русский и с казахского на русский (https://github.com/Intelligent-Systems-Phystech/psad-19/tree/master/labs/lab1/lab1_data/mt)
Стандартная оценка качества перевода производится с использованием специальной метрики BLEU.


Требуется определить для каких пар языков можно выделить какого-либо лидера среди переводчиков. Для BLEU учитывать только 
слова, регистр не учитывать.

## Формат данных
Названиие файлов имеет формат lang1_lang2_translation.txt:
 
 * lang_1, lang_2 --- языки (перевод с lang_1 на lang_2)
 * translation - какой переводчик использовался. gold - эталонный вариант, с которым сравнивается перевод от систем машинного перевода.

## Ссылки
* https://en.wikipedia.org/wiki/BLEU
* https://www.nltk.org/api/nltk.translate.html#module-nltk.translate.bleu_score
* https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer

# Импорт основных библиотек

In [95]:
import numpy as np
import math as m
import matplotlib.pylab as plt
import pandas as pd
import scipy.stats as st
import itertools
import math
import nltk
import glob
import nltk.translate.bleu_score as score
from statsmodels.stats.descriptivestats import sign_test
from nltk.tokenize import RegexpTokenizer
from permute.core import two_sample
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)

# Часть 1. Подготовка данных для статистического анализа.

Сначала разберёмся как пользоваться функцией $sentence\_bleu$ из модуля $nltk.translate.bleu\_score$. Источник информации: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/. Функция принимает на вход два списка - список справочных переводов предложений list(list(str)) ($reference$), а также предложение-кандидат на перевод list(str) ($candidate$). Функция $sentence\_bleu$ вычисляет метрику-BLEU между $candidate$ и $reference$.

In [44]:
reference = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
bleu = score.sentence_bleu(reference, candidate)
print(bleu)

0.7506238537503395


Будем пользоваться классом $RegexpTokenizer$, который предоставляет библиотека $nltk$, для того, чтобы можно было с помощью регулярных выражений по предложению получать список слов, состоящих только из буквенных и цифровых символов, и не учитывать знаки пунктуации. Источник информации: ссылка в описании задания - https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer.

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')

['Eighty', 'seven', 'miles', 'to', 'go', 'yet', 'Onward']

Создадим словарь $text\_transl$, который будет иметь $3$ ключа. Первый ключ - язык $lang\_from$, с которого переводится текст, второй ключ - язык $lang\_to$, на который переводится текст, третий - тип переводчика $type\_transl$. Данными в словаре будет набор предложений, которые даёт переводчик типа $type\_transl$ при переводе с языка $lang\_from$ на язык $lang\_to$. При этом все предложения разбиваются на слова с помощью $tokenizer$, а затем приводятся к нижнему регистру, так как в условии сказано: "Для BLEU учитывать только слова, регистр не учитывать". Для считывания файлов из текущей директории воспользуемся модулем $glob$. Также создадим списки для названий языков, с которых происходит перевод, как $arr\_lang\_from$; список языков, на которые происходит перевод, как $arr\_lang\_to$; типы переводчиков, как $arr\_type\_trans$.

In [23]:
set_lang_from = set()
set_lang_to = set()
set_type_trans = set()
text_transl = {}
for file_name in glob.glob("*.txt"):
    file_short_name = file_name[:-4]
    arr_file_name = file_short_name.split('_')
    lang_from = arr_file_name[0]
    set_lang_from.add(lang_from)
    lang_to = arr_file_name[1]
    set_lang_to.add(lang_to)
    type_trans = arr_file_name[2]
    set_type_trans.add(type_trans)
    print("file_name = {0} was opened".format(file_name))
    print("lang_from = {0}".format(lang_from))
    print("lang_to = {0}".format(lang_to))
    print("type_transl = {0}".format(type_trans))
    file = open(file_name)
    string_arr = []
    for string in file:
        add_string = tokenizer.tokenize(string)
        add_string = [x.lower() for x in add_string]
        string_arr.append(add_string)
    text_transl.update({(lang_from, lang_to, type_trans): string_arr})
    #text = tokenizer.tokenize(text)
    #text = [x.lower() for x in text]
    #print(text)
    #text_transl.update({(lang_from, lang_to, type_trans): text})
    file.close()
    print("file_name = {0} was closed".format(file_name))  

file_name = en_de_gold.txt was opened
lang_from = en
lang_to = de
type_transl = gold
file_name = en_de_gold.txt was closed
file_name = en_de_n.txt was opened
lang_from = en
lang_to = de
type_transl = n
file_name = en_de_n.txt was closed
file_name = en_de_orig.txt was opened
lang_from = en
lang_to = de
type_transl = orig
file_name = en_de_orig.txt was closed
file_name = en_de_p.txt was opened
lang_from = en
lang_to = de
type_transl = p
file_name = en_de_p.txt was closed
file_name = en_de_z.txt was opened
lang_from = en
lang_to = de
type_transl = z
file_name = en_de_z.txt was closed
file_name = en_ru_gold.txt was opened
lang_from = en
lang_to = ru
type_transl = gold
file_name = en_ru_gold.txt was closed
file_name = en_ru_n.txt was opened
lang_from = en
lang_to = ru
type_transl = n
file_name = en_ru_n.txt was closed
file_name = en_ru_orig.txt was opened
lang_from = en
lang_to = ru
type_transl = orig
file_name = en_ru_orig.txt was closed
file_name = en_ru_p.txt was opened
lang_from = en
la

In [24]:
arr_lang_from = list(set_lang_from)
arr_lang_to = list(set_lang_to)
arr_type_trans = list(set_type_trans)
arr_type_trans.remove('gold')
arr_type_trans.remove('orig')
print(arr_lang_from)
print(arr_lang_to)
print(arr_type_trans)

['en', 'kk']
['ru', 'de']
['p', 'z', 'n']


In [50]:
len(text_transl[('en', 'ru', 'n')])

10

In [54]:
sentence_test = text_transl[('en', 'ru', 'n')][1]

In [58]:
sentence_test

['в',
 'последнее',
 'время',
 'мы',
 'работаем',
 'по',
 'нескольким',
 'зарубежным',
 'схемам',
 'как',
 'в',
 'государственном',
 'так',
 'и',
 'в',
 'частном',
 'секторе',
 'развиваем',
 'сотрудничество',
 'не',
 'только',
 'с',
 'чешскими',
 'но',
 'и',
 'с',
 'иностранными',
 'финансовыми',
 'субъектами']

In [55]:
sentence_orig = text_transl[('en', 'ru', 'gold')][1]

Наконец-то, по полученным наборам предложений для каждого набора $(arr\_lang\_from, arr\_lang\_to, arr\_type\_trans)$. посчитаем метрику BLEU для каждого предложения из массива предложений $text\_transl[(arr\_lang\_from, arr\_lang\_to, arr\_type\_trans)]$.

In [47]:
dict_bleu = {}
for lang_from in arr_lang_from:
    for lang_to in arr_lang_to:
        for type_trans in arr_type_trans:
            if ((lang_from, lang_to, type_trans) in text_transl.keys()):
                sentences = text_transl[(lang_from, lang_to, type_trans)]
                bleu_arr = []
                for i in range(len(sentences)):
                    sentence = sentences[i]
                    true_sentence = text_transl[(lang_from, lang_to, 'gold')][i]
                    bleu = score.sentence_bleu([true_sentence], sentence)
                    bleu_arr.append(bleu)
                dict_bleu.update({(lang_from, lang_to, type_trans): bleu_arr})

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [51]:
dict_bleu[('en', 'ru', 'n')]

[6.5806869883189804e-155,
 0.3148282689155186,
 4.535506046608914e-155,
 4.429302865395896e-155,
 0.14440028187544326,
 3.976144999995337e-78,
 0.5900468726392808,
 0.360056585428503,
 1.1540791471212467e-231,
 1.2183324802375697e-231]

In [56]:
score.sentence_bleu([sentence_orig], sentence_test)

0.3148282689155186

Теперь выведем для каждой пары языков массивы метрики BLEU для каждого переводчика.

In [68]:
arr_type_trans

['p', 'z', 'n']

In [65]:
en_de_arr = []
for type_trans in arr_type_trans:
    en_de_arr.append(dict_bleu[('en', 'de', type_trans)])
en_ru_arr = []
for type_trans in arr_type_trans:
    en_ru_arr.append(dict_bleu[('en', 'ru', type_trans)])
kk_ru_arr = []
for type_trans in arr_type_trans:
    kk_ru_arr.append(dict_bleu[('kk', 'ru', type_trans)])

# Пара языков английский->немецкий

In [81]:
dict_for_data = {'p': en_de_arr[0], 'z': en_de_arr[1], 'n': en_de_arr[2]}
df = pd.DataFrame(data=dict_for_data)
df

,p,z,n
0,3.100712e-01,1.769498e-01,4.293663e-01
1,2.055549e-78,1.048628e-231,2.514947e-78
2,5.196599e-01,2.381949e-01,4.734491e-01
3,4.641609e-78,4.422441e-78,3.435175e-78
4,8.673017e-155,1.304856e-231,3.232426e-78
5,1.318331e-01,1.312155e-01,1.312097e-01
6,1.705957e-01,7.236349e-155,1.994045e-01
7,3.125580e-78,1.322613e-231,8.388266e-155
8,9.064563e-155,5.233428e-155,5.554838e-155
9,2.794226e-78,4.902402e-155,5.115747e-155


# Пара языков английский->русский

In [83]:
dict_for_data = {'p': en_ru_arr[0], 'z': en_ru_arr[1], 'n': en_ru_arr[2]}
df = pd.DataFrame(data=dict_for_data)
df

,p,z,n
0,3.145560e-01,7.855727e-155,6.580687e-155
1,4.217019e-01,1.475257e-01,3.148283e-01
2,6.290202e-155,1.195116e-231,4.535506e-155
3,2.223002e-78,2.293243e-78,4.429303e-155
4,2.171185e-01,8.839374e-02,1.444003e-01
5,5.607479e-155,9.918892e-232,3.976145e-78
6,5.331675e-01,5.859059e-01,5.900469e-01
7,3.082628e-01,2.592171e-01,3.600566e-01
8,1.154079e-231,1.311287e-231,1.154079e-231
9,6.968148e-155,3.634616e-78,1.218332e-231


# Пара языков казахский->русский

In [86]:
dict_for_data = {'p': kk_ru_arr[0], 'z': kk_ru_arr[1], 'n': kk_ru_arr[2]}
df = pd.DataFrame(data=dict_for_data)
df

,p,z,n
0,1.000938e-231,1.384293e-231,5.087741e-155
1,1.218332e-231,1.148419e-231,1.218332e-231
2,4.316700e-01,1.000000e+00,1.000000e+00
3,7.320648e-78,1.000000e+00,5.614022e-78
4,7.600394e-155,1.288230e-231,1.153142e-154
5,7.364817e-01,1.000000e+00,8.824969e-01
6,1.000000e+00,1.000000e+00,8.286572e-155
7,2.958026e-78,2.893292e-78,3.326840e-78
8,6.037949e-155,2.514086e-78,1.933314e-01
9,2.242387e-01,6.747523e-155,3.971419e-78


# Часть 2. Статистическая обработка полученных данных.

Сначала попробуем для каждой пары языков и для каждой пары переводчиков применить двухсторонний перестановочный критерий для того, чтобы посмотреть, можно ли в какой-то паре переводчиков на какой-то паре языков выделить лидера.

In [94]:
print("en_de for p and z, p_value = {0}".format(two_sample(en_de_arr[0], en_de_arr[1], alternative='two-sided')[0]))
print("en_de for p and n, p_value = {0}".format(two_sample(en_de_arr[0], en_de_arr[2], alternative='two-sided')[0]))
print("en_de for n and z, p_value = {0}".format(two_sample(en_de_arr[1], en_de_arr[2], alternative='two-sided')[0]))

en_de for p and z, p_value = 0.3921
en_de for p and n, p_value = 0.91402
en_de for n and z, p_value = 0.30811999999999995


In [93]:
print("en_ru for p and z, p_value = {0}".format(two_sample(en_ru_arr[0], en_ru_arr[1], alternative='two-sided')[0]))
print("en_ru for p and n, p_value = {0}".format(two_sample(en_ru_arr[0], en_ru_arr[2], alternative='two-sided')[0]))
print("en_ru for n and z, p_value = {0}".format(two_sample(en_ru_arr[1], en_ru_arr[2], alternative='two-sided')[0]))

en_ru for p and z, p_value = 0.42852
en_ru for p and n, p_value = 0.68396
en_ru for n and z, p_value = 0.7038599999999999


In [92]:
print("kk_ru for p and z, p_value = {0}".format(two_sample(kk_ru_arr[0], kk_ru_arr[1], alternative='two-sided')[0]))
print("kk_ru for p and n, p_value = {0}".format(two_sample(kk_ru_arr[0], kk_ru_arr[2], alternative='two-sided')[0]))
print("kk_ru for n and z, p_value = {0}".format(two_sample(kk_ru_arr[1], kk_ru_arr[2], alternative='two-sided')[0]))

kk_ru for p and z, p_value = 0.3886400000000001
kk_ru for p and n, p_value = 0.83476
kk_ru for n and z, p_value = 0.381


Как видим, к сожалению, нам не повезло и все полученные $p-value$ значительно больше уровня значимости $\alpha = 0.05$. Неудивительно, наших данных очень мало - всего $10$ предложений в каждом тексте. 

Попробуем применить знаковый критерий для массивов вида $\frac{arr_1}{arr_1 + arr_2}$, где отношение массивов понимается поэлементно, с параметром $\mu_0 = 0.5$

In [111]:
en_de_p = np.array(en_de_arr[0])
en_de_z = np.array(en_de_arr[1])
en_de_n = np.array(en_de_arr[2])
en_ru_p = np.array(en_ru_arr[0])
en_ru_z = np.array(en_ru_arr[1])
en_ru_n = np.array(en_ru_arr[2])
kk_ru_p = np.array(kk_ru_arr[0])
kk_ru_z = np.array(kk_ru_arr[1])
kk_ru_n = np.array(kk_ru_arr[2])

en_de_pz = en_de_p + en_de_z 
en_de_zn = en_de_z + en_de_n
en_de_pn = en_de_p + en_de_n
en_ru_pz = en_ru_p + en_ru_z 
en_ru_zn = en_ru_z + en_ru_n
en_ru_pn = en_ru_p + en_ru_n
kk_ru_pz = kk_ru_p + kk_ru_z 
kk_ru_zn = kk_ru_z + kk_ru_n
kk_ru_pn = kk_ru_p + kk_ru_n

In [112]:
print("sign-test, en_de, p and z, p_value = {0}".format(sign_test(en_de_p/en_de_pz, mu0=0.5)[1]))
print("sign-test, en_de, n and z, p_value = {0}".format(sign_test(en_de_n/en_de_zn, mu0=0.5)[1]))
print("sign-test, en_de, p and n, p_value = {0}".format(sign_test(en_de_p/en_de_pn, mu0=0.5)[1]))

sign-test, en_de, p and z, p_value = 0.001953125
sign-test, en_de, n and z, p_value = 0.10937500000000003
sign-test, en_de, p and n, p_value = 0.7539062500000002


In [113]:
print("sign-test, en_ru, p and z, p_value = {0}".format(sign_test(en_ru_p/en_ru_pz, mu0=0.5)[1]))
print("sign-test, en_ru, n and z, p_value = {0}".format(sign_test(en_ru_n/en_ru_zn, mu0=0.5)[1]))
print("sign-test, en_ru, p and n, p_value = {0}".format(sign_test(en_ru_p/en_ru_pn, mu0=0.5)[1]))

sign-test, en_ru, p and z, p_value = 0.7539062500000002
sign-test, en_ru, n and z, p_value = 0.7539062500000002
sign-test, en_ru, p and n, p_value = 0.5078125


In [114]:
print("sign-test, kk_ru, p and z, p_value = {0}".format(sign_test(kk_ru_p/kk_ru_pz, mu0=0.5)[1]))
print("sign-test, kk_ru, n and z, p_value = {0}".format(sign_test(kk_ru_n/kk_ru_zn, mu0=0.5)[1]))
print("sign-test, kk_ru, p and n, p_value = {0}".format(sign_test(kk_ru_p/kk_ru_pn, mu0=0.5)[1]))

sign-test, kk_ru, p and z, p_value = 1.0
sign-test, kk_ru, n and z, p_value = 0.5078125
sign-test, kk_ru, p and n, p_value = 0.5078125


Видим, что в паре переводчиков $p$ и $z$ на паре языков английский-немецкий есть явный лидер на уровне значимости $0.05$. На паре языков английский-немецкий для пары переводчиков $p$ и $n$ мы не можем опровергнуть гипотезу о том, что лидера нет. Если увеличить уровень значимости $\alpha$ до $0.11$, то в паре переводчиков $n$ и $z$ на этой же паре языков можем утверждать, что лидер есть. Таким образом, на уровне значимости значимости $\alpha = 0.11$ остаётся два варианта: $p \approx n < z$ или $p \approx n > z$.  Попробуем понять какой вариант более вероятен, а именно посмотрим насколько сумма массива $\frac{en\_de\_p}{en\_de\_p + en\_de\_z}$ отличается от величины $10 \cdot \frac{1}{2} = 5$ в случае, если бы переводчики были равны.

In [115]:
np.sum(en_de_p/en_de_pz)

7.969606167045871

In [116]:
np.sum(en_de_n/en_de_zn)

7.336156038993996

In [117]:
np.sum(en_de_p/en_de_pn)

5.54931902794926

На основании этих чисел есть основания полагать, что в паре $p$ и $z$ лидером является $p$ (мы не утверждаем это строго!), поэтому мы склоняемся к варианту $p \approx n > z$, то есть мы нашли не лидера, а лузера =(

В этой части кода мы пытались рассмотреть немного другую задачу, где сравнивали переводчики не на какой-то конкретной паре языков, а в целом на всех языках. Для этого мы соединили массивы, соответствующие каждому переводчику, между собой и применил знаковый критерий, аналогично тому, как мы делали это выше. 

In [98]:
arr_p = en_de_arr[0] + en_ru_arr[0] + kk_ru_arr[0]
arr_z = en_de_arr[1] + en_ru_arr[1] + kk_ru_arr[1]
arr_n = en_de_arr[2] + en_ru_arr[2] + kk_ru_arr[2]
arr_p = np.array(arr_p)
arr_z = np.array(arr_z)
arr_n = np.array(arr_n)
arr_pz = arr_p + arr_z
arr_pn = arr_p + arr_n
arr_zn = arr_z + arr_n
print("sign-test, p and z, p_value = {0}".format(sign_test(arr_p/arr_pz, mu0=0.5)[1]))
print("sign-test, p and n, p_value = {0}".format(sign_test(arr_p/arr_pn, mu0=0.5)[1]))
print("sign-test, n and z, p_value = {0}".format(sign_test(arr_n/arr_zn, mu0=0.5)[1]))

sign-test, p and z, p_value = 0.06142834573984148
sign-test, p and n, p_value = 0.8505540192127226
sign-test, n and z, p_value = 0.06142834573984148


In [101]:
np.sum(arr_p/arr_pz)

19.06667818310387

In [109]:
np.sum(arr_p/arr_pn)

16.42973262102849

In [102]:
np.sum(arr_n/arr_zn)

18.662918314692504

In [110]:
print("p_value for p and n = {0}".format(two_sample(arr_p, arr_n, alternative='two-sided')[0]))
print("p_value for p and z = {0}".format(two_sample(arr_p, arr_z, alternative='two-sided')[0]))
print("p_value for z and n = {0}".format(two_sample(arr_z, arr_n, alternative='two-sided')[0]))

p_value for p and n = 0.77442
p_value for p and z = 0.9010400000000001
p_value for z and n = 0.71154
